In [ ]:
# 1.5 wisker değerine göre her bir gruba ait outlier olabileceğini değerlendirdiğimiz gözlemleri tespit ediyoruz.

total_outliers = []

for model in df.make_model.unique():
    
    car_prices = df[df["make_model"]== model]["price"]
    
    Q1 = car_prices.quantile(0.25)
    Q3 = car_prices.quantile(0.75)
    IQR = Q3-Q1
    lower_lim = Q1-1.5*IQR
    upper_lim = Q3+1.5*IQR
    
    count_of_outliers = (car_prices[(car_prices < lower_lim) | (car_prices > upper_lim)]).count()
    
    total_outliers.append(count_of_outliers)
    
    print(f" The count of outlier for {model:<15} : {count_of_outliers:<5}, \
          The rate of outliers : {(count_of_outliers/len(df[df['make_model']== model])).round(3)}")
print()    
print("Total_outliers : ",sum(total_outliers), "The rate of total outliers :", (sum(total_outliers)/len(df)).round(3))

In [ ]:
from yellowbrick.classifier import ClassPredictionError
visualizer = ClassPredictionError(pipe_model)
# Fit the training data to the visualizer
visualizer.fit(X_train, y_train)
# Evaluate the model on the test data
visualizer.score(X_test, y_test)
# Draw visualization
visualizer.poof();
#classpredictionerror fonksiyonu class bazında yapılan hataları gösteriyor.

In [ ]:
# Veri setinin genel bir profilini elde etmek için
# Gözlem sayısı-veri tipi-null değer sayısı-unique değer sayısı-min-max değerleri
# null değerler "0" gözüküyor!

def summary(df, pred=None):
    obs = df.shape[0]
    Types = df.dtypes
    Counts = df.apply(lambda x: x.count())
    Min = df.min()
    Max = df.max()
    Uniques = df.apply(lambda x: x.unique().shape[0])
    Nulls = df.apply(lambda x: x.isnull().sum())
    print('Data shape:', df.shape)

    if pred is None:
        cols = ['Types', 'Counts', 'Uniques', 'Nulls', 'Min', 'Max']
        str = pd.concat([Types, Counts, Uniques, Nulls, Min, Max], axis = 1, sort=True)

    str.columns = cols
    print('___________________________\nData Types:')
    print(str.Types.value_counts())
    print('___________________________')
    return str

summary(df)

In [ ]:
# Veri setindeki yinelenen gözlemleri kontrol eder ve bunları kaldırır

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")

In [ ]:
features = pipe_model["OrdinalEncoder"].get_feature_names_out()
features

new_features = [i.replace("onehotencoder__","").replace("remainder__", "") for i in features]
new_features


model = GradientBoostingClassifier(learning_rate=0.5, max_features=5, n_estimators=300,
                           random_state=42)
operations = [("transformer", transformer), ("gb", model)]
pipe_model = Pipeline(steps=operations)
# Fit the pipeline
pipe_model.fit(X_train, y_train)

# Access feature importances from the GradientBoostingClassifier
gb_model = pipe_model.named_steps['gb']  # Access the GradientBoostingClassifier step
feature_importances = gb_model.feature_importances_

# Get feature names from the transformer
feature_names = pipe_model.named_steps['transformer'].get_feature_names_out()

# Create a DataFrame for feature importances
feats = pd.DataFrame(index=feature_names, data=feature_importances, columns=['gb_feature_importance'])

# Sort the features by importance
grad_imp_feats = feats.sort_values("gb_feature_importance", ascending=False)

# Display the sorted feature importances
print(grad_imp_feats)
